<a href="https://colab.research.google.com/github/btg1631/study_data_analytics/blob/main/docs/quests/EDAs/preprocessing_titanic_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
import seaborn as sns
import matplotlib.pyplot as plt

In [1]:
import pandas as pd
df_TFD = pd.read_csv("TitanicFromDisaster_train.csv")
df_TFD[:2]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


### 결측치 확인

In [2]:
df_TFD.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [3]:
df_TFD.isnull().sum()  # 각 컬럼별로 결측치 총합 표시

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

### 결측치 처리

In [ ]:
df_TFD.describe()

#### 1. 'Age' 열에서 결측치를 평균 나이로 채우기.

In [41]:
mean = df_TFD.describe().loc['mean', 'Age']  # 평균 = 29

In [42]:
df_Age = df_TFD.copy()
df_Age['Age'].fillna(mean, inplace=True)
df_Age.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

#### 2. 'Cabin' 열에 결측치가 있는 행을 찾고, 해당 행들의 평균 'Fare' 계산하기.


In [43]:
df_Cabin = df_TFD.copy()
mean_fare = df_Cabin[df_Cabin['Cabin'].isnull()]
mean_fare[:3]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.250,NaN,S
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.925,NaN,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.050,NaN,S


In [44]:
mean_fare['Fare'].mean()  # 평균 = 19

19.157325327510915

#### 3. 'Embarked' 열에서 결측치가 있는 행을 찾고, 가장 많이 등장하는 항구로 채우기.


In [26]:
df_Embarked = df_TFD.copy()
df_Embarked["Embarked"].value_counts()  # 가장 많이 등장하는 항구 = S

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [28]:
df_Embarked.fillna('S', inplace=True)
df_Embarked.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

#### 4. 'Fare' 열에서 평균보다 3표준편차(mean + 3 * std) 이상 높은 이상치를 가진 행들을 찾기.


In [67]:
df_Fare = df_TFD.copy()
mean = df_Fare['Fare'].mean()
std = df_Fare['Fare'].std()

In [69]:
result = mean+3*std
result

181.2844937601173

In [ ]:
sns.boxplot(data=df_Fare, x='Fare')

In [ ]:
fare = f'Fare >= {result}'
outlier_sibsp = df_Fare.query(fare)
outlier_sibsp

#### 5. 'SibSp' (형제 자매/배우자 수)가 5 이상인 이상치를 가진 행들을 찾기.


In [ ]:
df_SibSp = df_TFD.copy()
sns.boxplot(data=df_SibSp, x='SibSp')

In [ ]:
outlier_sibsp = 'SibSp >= 5'
result = df_SibSp.query(outlier_sibsp)
result

#### 6. 'Parch' (부모/자녀 수) 열에서 0이 아닌 값을 가진 행들의 평균 'Age' 계산하기.

In [73]:
df_Parch = df_TFD.copy()
Parch_not_zero = 'Parch != 0'
result_Parch = df_Parch.query(Parch_not_zero)

In [74]:
result_Parch['Age'].mean()  # 평균 = 23

23.006062176165802

#### 7. 'Fare'가 0인 행들을 찾고, 이를 평균 'Fare'로 채우기.


In [ ]:
df_Fare = df_TFD.copy()
mean_fare = df_Fare['Fare'].mean()  # 평균 = 32
df_Fare.loc[df_Fare['Fare'] == 0, 'Fare'] = mean_fare
df_Fare[['Fare']]

#### 8. 'Pclass' (티켓 등급)가 1등급인 승객들 중 'Cabin' 열의 결측치 비율 계산하기.


In [96]:
df_Pclass = df_TFD.copy()
Pclass_first = 'Pclass == 1'
result_Pclass = df_Pclass.query(Pclass_first)

In [ ]:
result_Pclass['Cabin'].info()  # 전체 216, 결측치 40

In [97]:
missing_Cabin = result_Pclass['Cabin'].isnull().sum()
total = result_Pclass['Cabin'].count()

In [102]:
result = (missing_Cabin/total) * 100
result  # 22%

22.727272727272727

#### 9. 'Age'와 'Fare' 모두에 결측치가 있는 행들을 찾기.


In [131]:
df_AgeFare = df_TFD.copy()
df_AgeFare[(df_AgeFare['Age'].isnull()) & (df_AgeFare['Fare']==0)]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
277,278,0,2,"Parkes, Mr. Francis ""Frank""",male,NaN,0,0,239853,0.0,NaN,S
413,414,0,2,"Cunningham, Mr. Alfred Fleming",male,NaN,0,0,239853,0.0,NaN,S
466,467,0,2,"Campbell, Mr. William",male,NaN,0,0,239853,0.0,NaN,S
481,482,0,2,"Frost, Mr. Anthony Wood ""Archie""",male,NaN,0,0,239854,0.0,NaN,S
633,634,0,1,"Parr, Mr. William Henry Marsh",male,NaN,0,0,112052,0.0,NaN,S
674,675,0,2,"Watson, Mr. Ennis Hastings",male,NaN,0,0,239856,0.0,NaN,S
732,733,0,2,"Knight, Mr. Robert J",male,NaN,0,0,239855,0.0,NaN,S
815,816,0,1,"Fry, Mr. Richard",male,NaN,0,0,112058,0.0,B102,S


#### 10.'Sex' 열에서 값이 'male' 또는 'female'이 아닌 이상치를 가진 행들을 찾기.

In [110]:
df_TFD['Sex'].unique()  # 이상치 없음

array(['male', 'female'], dtype=object)

In [116]:
df_Sex = df_TFD.copy()
sex_male = 'Sex != "male"'
sex_female = 'Sex != "female"'
outlier_both = f'{sex_male} and {sex_female}'

result = df_Sex.query(outlier_both)
result

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked


#### 11. 'Fare' 열의 결측치(값이 0)를 해당 'Pclass'의 평균 요금으로 채우기.